In [ ]:
import numpy as np
from kspec_metrology.analysis.findpeak import findpeak
from kspec_metrology.analysis.matchfiber import matchfiber
from kspec_metrology.analysis.fitdistortion import fitdistortion
from kspec_metrology.analysis.utils import transform, find_angle_double_method2_elbow_down
from astropy.io import ascii, fits
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
import json

dtor = np.pi / 180.
rtod = 180. / np.pi

In [ ]:
tab = ascii.read('Fiber_Configuration_250415.txt')
tab.rename_columns(tab.colnames[:4], ["ID", "X", "Y", "FiducialFlag"])

ids = [
       'B5' , 'C2', 'D0',
       'E5' , 'E8', 'G4',
       'G11', 'H7', 'I2',
       'I9' , 'K3', 'K6',
       'L10', 'M7'
      ]


fid_ids = tab["ID"][tab["FiducialFlag"] == 1]
pick_ids = list(dict.fromkeys(list(ids) + list(fid_ids)))

remove_list = ['A0', 'Z1', 'Z4', 'Z10']
for item in remove_list:
    pick_ids.remove(item)

xy_map = {r["ID"]: (r["X"], r["Y"]) for r in tab}
xy = np.array([xy_map[i] for i in pick_ids], dtype=float)
xorigin, yorigin = xy[:, 0], xy[:, 1]

x, y = np.copy(xorigin), np.copy(yorigin)
fid_flag = np.zeros(x.size, dtype=bool)
fid_flag[14:] = True

#Calibration ---------------------------------------------------------------
alpha_goal = 18.*dtor
beta_goal = 70.*dtor

x[~fid_flag] += (5.2*np.cos(alpha_goal) + 11.6*np.cos(alpha_goal+beta_goal))
y[~fid_flag] += (5.2*np.sin(alpha_goal) + 11.6*np.sin(alpha_goal+beta_goal))

#Read target ---------------------------------------------------------------
#with open('./target/object.info', 'r') as ff:
#    target = json.load(ff)
#    xall, yall = target['xp'], target['yp']
#x[:14], y[:14] = xall[:14], yall[:14]

theta_true = np.zeros(14); phi_true = np.copy(theta_true)
for i in range(14):
    theta_true[i], phi_true[i] = find_angle_double_method2_elbow_down(xorigin[i], yorigin[i], x[i], y[i])

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))

ax.scatter(xorigin, yorigin, c='k', marker='.')
ax.scatter(x[~fid_flag], y[~fid_flag], c='r', marker='*')
for i in range(14):
    xa = xorigin[~fid_flag][i]+ 5.2*np.cos(theta_true[i])
    ya = yorigin[~fid_flag][i]+ 5.2*np.sin(theta_true[i])
    ax.plot([xorigin[~fid_flag][i], xa], [yorigin[~fid_flag][i], ya], c='r', lw=1)

    xb = xa + 11.6*np.cos(theta_true[i] + phi_true[i])
    yb = ya + 11.6*np.sin(theta_true[i] + phi_true[i])
    ax.plot([xa, xb], [ya, yb], c='b', lw=1)

In [ ]:
npeaks = fid_flag.sum()
_, xobs, yobs = findpeak(npeaks, data_dir='./data/', head='Fiducial_Trial1_', threshold=1e3)
imatch, theta_guess, (xoff_guess, yoff_guess) = matchfiber(x[fid_flag], y[fid_flag], xobs, yobs, nbuffer=20)
_, _, dx, dy, _, _, _, _, inv_popt_obs = fitdistortion(x[fid_flag], y[fid_flag], fid_flag[fid_flag]
                                                          , xobs, yobs
                                                          , xorigin[fid_flag], yorigin[fid_flag]
                                                          , imatch, theta_guess, xoff_guess, yoff_guess)

dr = np.sqrt(dx**2 + dy**2)
print(dr*1e3)

In [ ]:
npeaks = (~fid_flag).sum()
_, xobs, yobs = findpeak(npeaks, data_dir='./data/', head='Fiber_Trial_Low7_', threshold=2e3)

xtemp, ytemp = transform(xobs, yobs, inv_popt_obs)
tree = cKDTree(np.vstack((xtemp, ytemp)).T)
dd, ii = tree.query(np.vstack((x[~fid_flag], y[~fid_flag])).T, k=1)

xfib, yfib = xtemp[ii], ytemp[ii]

theta_obs = np.zeros(xfib.size); phi_obs = np.copy(theta_obs)
for i in range(xfib.size):
    theta_obs[i], phi_obs[i] = find_angle_double_method2_elbow_down(xorigin[i], yorigin[i], xfib[i], yfib[i])
    theta_new, phi_new = (theta_true[i] - theta_obs[i]), (phi_true[i] - phi_obs[i])

    theta_print, phi_print = theta_new%(2.*np.pi), phi_new%(np.pi)
    if theta_print > np.pi: theta_print -= 2.*np.pi
    if phi_print > np.pi/2.: phi_print -= np.pi

    dist = np.sqrt((xfib[i]-x[~fid_flag][i])**2 + (yfib[i]-y[~fid_flag][i])**2)

    print(f"Positioner {i+1} - Alpha: {theta_print*rtod:.2f}, Beta: {phi_print*rtod:.2f}, Distance: {(dist*1e3):.1f}")

In [ ]:
iaxis = np.array([3, 6, 11, 9, 2, 4, 8, 12, 10, 5, 1, 13, 14, 7])-1

for kk,jj in enumerate(iaxis):
    theta_new, phi_new = (theta_true[jj] - theta_obs[jj]), (phi_true[jj] - phi_obs[jj])

    theta_print, phi_print = theta_new%(2.*np.pi), phi_new%(np.pi)
    if theta_print > np.pi: theta_print -= 2.*np.pi
    if phi_print > np.pi/2.: phi_print -= np.pi

    print(f"Axis {2*kk+1} : {(-theta_print*rtod):.2f}")
    print(f"Axis {2*kk+2} : {(-phi_print*rtod):.2f}")

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))

ax.scatter(xorigin, yorigin, c='k', marker='.')
ax.scatter(x[~fid_flag], y[~fid_flag], c='r', marker='*')
for i in range(xfib.size):
    xa = xorigin[~fid_flag][i]+ 5.2*np.cos(theta_true[i])
    ya = yorigin[~fid_flag][i]+ 5.2*np.sin(theta_true[i])
    ax.plot([xorigin[~fid_flag][i], xa], [yorigin[~fid_flag][i], ya], c='r', lw=1)

    xb = xa + 11.6*np.cos(theta_true[i] + phi_true[i])
    yb = ya + 11.6*np.sin(theta_true[i] + phi_true[i])
    ax.plot([xa, xb], [ya, yb], c='b', lw=1)

#ax.scatter(xfib, yfib, c='tab:orange', marker='*')
for i in range(xfib.size):
    xa = xorigin[~fid_flag][i]+ 5.2*np.cos(theta_obs[i])
    ya = yorigin[~fid_flag][i]+ 5.2*np.sin(theta_obs[i])
    ax.plot([xorigin[~fid_flag][i], xa], [yorigin[~fid_flag][i], ya], c='tab:orange', lw=1)

    xb = xa + 11.6*np.cos(theta_obs[i] + phi_obs[i])
    yb = ya + 11.6*np.sin(theta_obs[i] + phi_obs[i])
    ax.plot([xa, xb], [ya, yb], c='tab:blue', lw=1)

### Check Image

In [ ]:
im = fits.getdata('./data/Fiber_test_0.fits')

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

ax.imshow(np.log10(im), cmap='gray')